In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.metrics import RootMeanSquaredError
from keras.optimizers import AdamW
from keras import backend as K

In [2]:
X_train = np.load('../y_smp_train.npy')
y_train = np.load('../pars_smp_train.npy')
X_test = np.load('../y_smp_test.npy')

In [3]:
X_train.shape, y_train.shape

((100000, 200, 3), (100000, 15, 1))

In [7]:
def quantile_loss_75(y_true, y_pred):
    error = y_true - y_pred
    loss = K.mean(K.maximum(0.75 * error, (0.75 - 1) * error))
    return loss

In [8]:
time_steps = 200
num_features = 3

In [9]:
model = Sequential()

model.add(Conv2D(16, (3,3), activation='relu', input_shape=(time_steps, num_features, 1), padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (12,6), activation='relu', strides=(6, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', strides=(2, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(15))

opt = AdamW(learning_rate=0.0001)
model.compile(loss=quantile_loss_75, optimizer=opt, metrics=[RootMeanSquaredError()])

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 200, 3, 16)        160       
                                                                 
 up_sampling2d_2 (UpSamplin  (None, 400, 6, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 400, 6, 32)        4640      
                                                                 
 up_sampling2d_3 (UpSamplin  (None, 800, 12, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 132, 7, 32)        73760     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 66, 4, 32)        

In [7]:
model.fit(X_train, y_train, batch_size=128, epochs=25)

Epoch 1/25


2023-10-15 04:40:23.182003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900
2023-10-15 04:40:23.883598: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-15 04:40:23.924872: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f7af3e2d240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-15 04:40:23.924899: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-10-15 04:40:23.929188: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-15 04:40:24.052561: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the pro

782/782 [==============================] - 13s 9ms/step - loss: 0.2470 - root_mean_squared_error: 0.8202
Epoch 2/25
782/782 [==============================] - 7s 9ms/step - loss: 0.1038 - root_mean_squared_error: 0.4701
Epoch 3/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0878 - root_mean_squared_error: 0.4004
Epoch 4/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0802 - root_mean_squared_error: 0.3693
Epoch 5/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0757 - root_mean_squared_error: 0.3508
Epoch 6/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0731 - root_mean_squared_error: 0.3397
Epoch 7/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0708 - root_mean_squared_error: 0.3306
Epoch 8/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0689 - root_mean_squared_error: 0.3219
Epoch 9/25
782/782 [==============================] - 7s 9ms/step - loss: 0.0672 - root_me

KeyboardInterrupt: 

In [ ]:
pred = model.predict(X_test)

In [ ]:
np.save('test_pred_75', pred)

In [ ]:
model.save("model_75.h5")